In [36]:
"""
author @ Cavin Gada
    Description:
    This project allows users to get historical data on a custom built stock portfolio (with tickers
    and corresponding shares, a start date, and an end date). A user can retrieve the semi deviation
    (downwards deviation), monthly volatility, value at risk (95% confidence), and maximum drawdown 
    with regards to their portfolio by calling the respective function.  
"""
# imports
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
from scipy.stats import norm

In [37]:
class Portfolio:

    def __init__(self, stocks, start, end):

        self.stockList = stocks
        self.startDate = start
        self.endDate = end

        # remember total shares to calculate weights
        self.totalShares = 0
        for stock in self.stockList:
            self.totalShares += self.stockList[stock]

        # decimal weight of each stock in portfolio
        self.stockWeights = {}
        for stock in stockList:
            self.stockWeights[stock] = self.stockList[stock]/self.totalShares
    
        
    def semiDeviation(self):

        """ returns the semideviation of the portfolio (by daily return rates).

            The implementation is as follows:
            1. Take a look at stock X's daily return rate data.
            2. sum the squared distance from each return rate (that is < the average) and the average.
            3. Divide this by the total number of return rates < the average.
            4. Take the square root of this value. 
            5. Multiply this by the stock's weight in the portfolio
            6. Add this value to the final semiDeviation. 
            7. Repeat for each stock Xi.  
        
        """
        semiDeviation = 0
        for stock in self.stockList:
            # retrieve stock's data.
            stock_df = web.DataReader([stock], 'yahoo', start = self.startDate, end = self.endDate)
            # retrieve stock's daily return rates
            returns = stock_df[("Close", stock)].pct_change(1)
            # retrieve the average of the daily return rates
            avg = returns.mean()
            # keep track of the stock's Semi Standard Deviation. 
            stockSSD = 0
            # keep track of the number of return rates below the average.
            numBelowAvg = 0
            # loop through each daily return rate
            for r in returns:
                # if the return rate is below the average, then take the squared distance from r and the average
                # and add it to the stockSSD. Increment the amount of return rates below the average by 1. 
                if r < avg:
                    stockSSD+=np.square(r-avg)
                    numBelowAvg+=1
            # divide the sum of squared distances by the total number of return rates below average and take the square root. 
            stockSSD = np.sqrt(stockSSD/numBelowAvg)
            # add the stock's semi standard deviation multiplied by its weight to the total semi deviation. 
            semiDeviation += stockSSD * self.stockWeights[stock]

        return semiDeviation

    def monthlyVolatility(self):
        totalMonthlyVolatility = 0
        for stock in self.stockList:

            # keep track of daily log returns in a list. 
            logreturns = []

            # keep track of total stock's monthly volatility
            stockVolatility = 0

            # retrieve stock's data.
            stock_df = web.DataReader([stock], 'yahoo', start = self.startDate, end = self.endDate)
            
            # retrieve stock's daily return rates (+1 to remove negatives)
            returns = stock_df[("Close", stock)].pct_change(1)+1

            # aquire all log returns and append to list
            for r in returns:
                logreturns+=[np.log(r)]

            # disclude the first log return (since it is NaN)
            # take standard deviation of all valid log returns and multiply
            # sqrt of trading days in a month (21)
            stockVolatility = np.std(logreturns[1:]) * np.sqrt(21)

            # multiply the stock's monthly volatility by its weight in the portfolio
            # and add to the total volatility. 
            totalMonthlyVolatility += stockVolatility * self.stockWeights[stock]

        return totalMonthlyVolatility

    def var(self):

        totalVaR = 0

        for stock in self.stockList:

            # retrieve stock's data.
            stock_df = web.DataReader([stock], 'yahoo', start = self.startDate, end = self.endDate)
            
            # retrieve stock's daily return rates
            returns = stock_df[("Close", stock)].pct_change(1)

            # retrieve daily return mean and standard deviation
            mean = np.mean(returns)
            sd = np.std(returns)

            # use normal density curve to estimate the value at risk given 95% confidence
            stockVaR = norm.ppf(.05, mean, sd)
            totalVaR += stockVaR * self.stockWeights[stock]

        return totalVaR
    
    def sortinoRatio(self):
        portfolioReturn = 0

        #risk free rate
        riskFreeRate = .0159 #10 year treasury yield.

        for stock in self.stockList:

            # retrieve stock's data.
            stock_df = web.DataReader([stock], 'yahoo', start = self.startDate, end = self.endDate)
            
            # retrieve stock's daily return rates
            returns = stock_df[("Close", stock)].pct_change(1)
            # add the mean of the stock's daily return rates multiplied by its weight in the portfolio to the total return of portfolio
            portfolioReturn+= np.mean(returns) * self.stockWeights[stock]

        # take the portfolio return and subtract the benchmark return. Then divide this by the downside (semi) deviation of the portfolio
        returnDiff = portfolioReturn - riskFreeRate
        return returnDiff/self.semiDeviation()
    
    def maxDrawDown(self):
        
        #initialize an empty series. This will be useful for adding each day's stock data. 
        portfolioSeries = pd.Series([],dtype="float64")

        # initialize an empty list to keep track of total portfolio returns per day. 
        # I choose to use a list here to make it easier to access specific indices of data. 
        portfolioReturns = []

        # the empty series can only be updated correctly if it is initially set to the first stock's data 
        # then added to the series of the other stocks' data. This is why we have a flag. 

        flag_first = True

        for stock in self.stockList:
            # retrieve stock's data.
            stock_df = web.DataReader([stock], 'yahoo', start = self.startDate, end = self.endDate)
            
            # retrieve stock's daily return rates
            returns = stock_df[("Close", stock)].pct_change(1)
            
            #if we are iterating through the first stock's data. 
            if flag_first:
                portfolioSeries = (returns*self.stockWeights[stock])
                flag_first=False
            #if we have already iterated through the first stock's data. 
            else:
                portfolioSeries += (returns*self.stockWeights[stock])
        
        #insert portfolio series data into the portfolio list. This will let us access indices with less complexity.

        portfolioReturns = portfolioSeries.tolist()[1:]
        # 1. find smallest relative minimum after the absolute maximum 
        # 2. find the largest relative maximum before the absolute minimum. 
        # 3. after calculating the drawdowns of the absolute max -> relative min and 
        #    relative max -> absolute min, return the smaller value (since both are negative
        #    and we wish to find the larger "drop")

        # instantiate absolute max, min and create variables to store local max, min. 
        # we initialize the local max to the absolute minimum and the local min to the absolute maximum
        # because the local max should always be at least the absolute minimum and the local min should always 
        # be at most the absolute maximum. 
        # also, keep track of index of absolute max and min to know where to loop from. 

        absoluteMax = max(portfolioReturns)
        absoluteMaxIndex = portfolioReturns.index(absoluteMax)
        localMin = absoluteMax

        absoluteMin = min(portfolioReturns)
        absoluteMinIndex = portfolioReturns.index(absoluteMin)
        localMax = absoluteMin

        # loop from the index of the absolute max element to the end of the portfolio. We want to find a drop
        # so we look for a local min AFTER the max. 

        for i in range(absoluteMaxIndex, len(portfolioReturns)):
            if portfolioReturns[i] < localMin:
                localMin = portfolioReturns[i]
        
        # loop from 0 to the index of the absolute min element. We want to find a drop so we look for a local max
        # BEFORE the min. 
        
        for i in range(0, absoluteMinIndex): 
            if portfolioReturns[i] > localMax:
                localMax = portfolioReturns[i]

        absoluteMaxToLocalMin = (localMin-absoluteMax)/(absoluteMax)
        localMaxToAbsoluteMin = (absoluteMin-localMax)/(localMax)

        return min(absoluteMaxToLocalMin, localMaxToAbsoluteMin)

In [38]:
# example portfolio
stockList = {"AAPL": 50, "GME": 150, "TSLA": 5, "AAL": 200, "AMZN": 1}
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2020, 1, 1)
p1 = Portfolio(stockList, start, end)

# Runtime: 28.8s. We can do better...
print("Portfolio's Semi Deviation is: " + str(round(p1.semiDeviation()*100, 3)) + "%")
print("Portfolio's Monthly volatility is " + str(round(p1.monthlyVolatility()*100,3)) + "%")
print("Portfolio's Sortino Ratio is " + str(round(p1.sortinoRatio(),3)))
print("Portfolio's 95% Value at Risk is " + str(round(p1.var()*100,3)) + "%")
print("Portfolio's maximum drawdown is " + str(round(p1.maxDrawDown()*100,3)) + "%")



Portfolio's Semi Deviation is: 2.98%
Portfolio's Monthly volatility is 13.508%
Portfolio's Sortino Ratio is -0.561
Portfolio's 95% Value at Risk is -4.786%
Portfolio's maximum drawdown is -204.819%
